# First test for the kaggle 

In [113]:
import numpy as np
import matplotlib.pyplot as plt
import pickle as pkl
import networkx as nx
import pandas as pd
from scipy import optimize
from scipy.linalg import cho_factor, cho_solve
from tqdm import tqdm

The evaluation pages describes how submissions will be scored and how students should format their submissions. The metric is the AUC (area under curve). The data contains 2 classes. 
## Submission
Format Submission files should contain two columns: Id and Prediction. The file should contain a header and have the format described below. Id represents the identifier of the test example, ranging from 1 to 2000. The prediction is the corresponding logit which is a real number Ex: ``` Id,Prediction 1, -1.1 2, 3.2 3, -2.4 4,-0.5 5,2.1 6,0.1 7,-0.9 ``` Below, you will also find a piece of code for reading/writing the data. ```python import pickle as pkl import pandas as pd with open('training_data.pkl', 'rb') as file: train_graphs = pkl.load(file) with open('test_data.pkl', 'rb') as file: test_graphs = pkl.load(file) with open('training_labels.pkl', 'rb') as file: train_labels = pkl.load(file) # define your learning algorithm here # for instance, define an object called ``classifier'' # classifier.train(train_labels,train_graphs) # predict on the test data # for instance, test_preds = classifier.predict(test_graphs) Yte = {'Prediction' : test_preds} dataframe = pd.DataFrame(Yte) dataframe.index += 1 dataframe.to_csv('test_pred.csv',index_label='Id') ```

In [169]:
with open('data/training_data.pkl', 'rb') as file: train_graphs = pkl.load(file)
with open('data/test_data.pkl', 'rb') as file: test_graphs = pkl.load(file)
with open('data/training_labels.pkl', 'rb') as file: train_labels = pkl.load(file)

In [109]:
new_train_labels = 2*train_labels-1

## nth order walk kernel

In [118]:
class walk_kernel:
    def __init__(self, order):
        self.order = order
        
    def similarity(self,G1,G2):
        # Input : G1,G2 two graphs
        # Output : K(G1,G2)
        product_graph = nx.cartesian_product(G1,G2)
        A = nx.adjacency_matrix(product_graph)
        n = A.shape[0]
        return (np.ones((1,n))@np.power(A,self.order)@np.ones((n,1)))[0,0]
    
    def kernel(self,X,Y):
        # Input : X vector of N graphs, Y vector of M graphs
        # Output : K similarity matrix between X and Y
        N = len(X)
        M = len(Y)
        K = np.zeros((N,M))
        for i in tqdm(range(N)):
            for j in range(i,M):
                res = self.similarity(train_graphs[i],train_graphs[j])
                K[i,j] = res
                K[j,i] = res
        return K

In [117]:
walk_kernel_1 = walk_kernel(1)

In [119]:
#N = len(train_graphs)
N = 60
K = walk_kernel_1.kernel(train_graphs[:N],train_graphs[:N])
        

/var/folders/_d/xgqzpq6x1dn227pxd6dvmhsc0000gn/T/ipykernel_9222/3148421536.py:9: FutureWarning: adjacency_matrix will return a scipy.sparse array instead of a matrix in Networkx 3.0.
  A = nx.adjacency_matrix(product_graph)


## Test of SVC

In [165]:
class KernelSVC:
    def __init__(self, C, kernel, epsilon = 1e-3):
        self.type = 'non-linear'
        self.C = C                               
        self.kernel = kernel        
        self.alpha = None
        self.support = None
        self.epsilon = epsilon
        self.norm_f = None
        self.X = None
        self.y = None
    
    def fit(self, X, y,K):
       #### You might define here any variable needed for the rest of the code
        self.X = X
        self.y = y
        N = len(y)
        #K = kernel(X,X)
        M = np.diag(y)@K@np.diag(y)
        # Lagrange dual problem
        def loss(alpha):
            return  (1/2)*alpha.T@M@alpha - np.sum(alpha)#'''--------------dual loss ------------------ '''

        # Partial derivate of Ld on alpha
        def grad_loss(alpha):
            return  M@alpha - np.ones_like(alpha)# '''----------------partial derivative of the dual loss wrt alpha -----------------'''

        # Constraints on alpha of the shape :
        # -  d - C*alpha  = 0
        # -  b - A*alpha >= 0
        fun_eq = lambda alpha: alpha.T@y # '''----------------function defining the equality constraint------------------'''        
        jac_eq = lambda alpha: y   #'''----------------jacobian wrt alpha of the  equality constraint------------------'''
        fun_ineq = lambda alpha: np.concatenate((C*np.ones_like(alpha) - alpha,alpha))  # '''---------------function defining the inequality constraint-------------------'''     
        jac_ineq = lambda alpha:  np.concatenate((-np.eye(N),np.eye(N))) # '''---------------jacobian wrt alpha of the  inequality constraint-------------------'''
        
        constraints = ({'type': 'eq',  'fun': fun_eq, 'jac': jac_eq},
                       {'type': 'ineq', 
                        'fun': fun_ineq , 
                        'jac': jac_ineq})

        optRes = optimize.minimize(fun=lambda alpha: loss(alpha),
                                   x0=np.ones(N), 
                                   method='SLSQP', 
                                   jac=lambda alpha: grad_loss(alpha), 
                                   constraints=constraints)
        self.alpha = optRes.x
        ## Assign the required attributes

        indices =  np.where((self.alpha>0) & (self.alpha<C))[0]
        self.margin_points = X[indices] #'''------------------- A matrix with each row corresponding to a point that falls on the margin ------------------'''
        self.b =  np.mean(y[indices] - self.separating_function(self.margin_points))#''' -----------------offset of the classifier------------------ '''
        self.norm_f = self.alpha.T@K@self.alpha # '''------------------------RKHS norm of the function f ------------------------------'''
        self.support = X[np.where(self.alpha>self.epsilon)[0]]
        
    ### Implementation of the separting function $f$ 
    def separating_function(self,x):
        # Input : matrix x of shape N data points times d dimension
        # Output: vector of size N
        return np.sum(np.diag(self.y*self.alpha)@kernel(self.X,x),axis=0)

    def predict(self, X):
        """ Predict y values in {-1, 1} """
        d = self.separating_function(X)
        return 2 * (d+self.b> 0) - 1

In [156]:
C=1
kernel = walk_kernel(1).kernel
X = train_graphs[:60]
y = new_train_labels[:60]
K = kernel(X,X)
model = KernelSVC(C=C, kernel=kernel)

  0%|          | 0/60 [00:00<?, ?it/s]/var/folders/_d/xgqzpq6x1dn227pxd6dvmhsc0000gn/T/ipykernel_9222/4105415923.py:9: FutureWarning: adjacency_matrix will return a scipy.sparse array instead of a matrix in Networkx 3.0.
  A = nx.adjacency_matrix(product_graph)
100%|██████████| 60/60 [00:15<00:00,  3.93it/s]


In [166]:
C = 1.
model = KernelSVC(C=C, kernel=kernel)
model.fit(np.array(X), np.array(y), K)

/var/folders/_d/xgqzpq6x1dn227pxd6dvmhsc0000gn/T/ipykernel_9222/3340371026.py:3: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  model.fit(np.array(X), np.array(y), K)
  0%|          | 0/60 [00:00<?, ?it/s]/var/folders/_d/xgqzpq6x1dn227pxd6dvmhsc0000gn/T/ipykernel_9222/4105415923.py:9: FutureWarning: adjacency_matrix will return a scipy.sparse array instead of a matrix in Networkx 3.0.
  A = nx.adjacency_matrix(product_graph)
100%|██████████| 60/60 [00:11<00:00,  5.33it/s]


In [167]:
pred = model.predict(X)

  0%|          | 0/60 [00:00<?, ?it/s]/var/folders/_d/xgqzpq6x1dn227pxd6dvmhsc0000gn/T/ipykernel_9222/4105415923.py:9: FutureWarning: adjacency_matrix will return a scipy.sparse array instead of a matrix in Networkx 3.0.
  A = nx.adjacency_matrix(product_graph)
100%|██████████| 60/60 [00:15<00:00,  3.98it/s]


In [168]:
pred

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

In [160]:
y

array([-1, -1, -1, -1,  1, -1, -1, -1, -1, -1, -1, -1, -1, -1,  1, -1, -1,
        1, -1,  1, -1, -1, -1, -1, -1, -1,  1, -1, -1,  1, -1, -1, -1,  1,
       -1, -1,  1, -1, -1,  1, -1,  1, -1,  1, -1, -1,  1,  1, -1,  1,  1,
       -1, -1, -1, -1, -1, -1,  1, -1,  1])